In [1]:
import os
from langchain_google_vertexai import ChatVertexAI
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool

In [4]:
# CONFIGURACIÓN DE CREDENCIALES
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../arch-dev-agent-87f23e12bec3.json"
os.environ["GOOGLE_CLOUD_PROJECT"] = "arch-dev-agent"
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"

print("Conectado a Vertex AI")

Conectado a Vertex AI


In [ ]:
# personalidad persuasiva
SALES_SYSTEM_PROMPT = """
ERES UN VENDEDOR EXPERTO, NO UN ASISTENTE. TU NOMBRE ES 'ALEX'.
Tu objetivo es CERRAR VENTAS.

REGLAS DE COMPORTAMIENTO:
1. PERSUASIÓN: Si el usuario duda del precio, justifica con calidad y beneficios a largo plazo.
2. URGENCIA: Menciona sutilmente que el stock es limitado si el usuario muestra interés.
3. CROSS-SELLING: Si compran zapatos, sugiere calcetines o limpiadores.
4. CIERRE: Siempre termina tus respuestas invitando a la compra. Ej: "¿Te los envío hoy mismo?".

NO HAGAS:
- No seas pasivo ("¿En qué te ayudo?"). Sé proactivo ("Tengo una oferta para ti").
- No inventes stock. Usa la herramienta 'consultar_stock'.

REGLAS DE HERRAMIENTAS:
- Si el usuario pregunta "¿Tienen X?", usa 'consultar_inventario'.
- Si el usuario dice "Lo quiero", "Dame 2", "Me lo llevo", NO dudes: usa 'crear_pedido' de inmediato.
"""

print("Personalidad de Vendedor cargada")

Personalidad de Vendedor cargada


In [ ]:
# herramientas simuladas para el prototipo
@tool
def consultar_inventario(producto_query: str):
    # Busca productos, precios y stock en la base de datos.
    # mock de simulación
    inventario_fake = {
        "nike": {"nombre": "Nike Air Zoom", "precio": 120, "stock": 3, "desc": "Ideal para correr"},
        "adidas": {"nombre": "Adidas Ultraboost", "precio": 140, "stock": 10, "desc": "Máximo confort"},
    }
    
    query = producto_query.lower()
    for key, data in inventario_fake.items():
        if key in query:
            return f"ENCONTRADO: {data}"
    return "NO ENCONTRADO en inventario."

@tool
def crear_pedido(producto: str, cantidad: int):
    """
    ÚSALA INMEDIATAMENTE cuando el usuario confirme que quiere comprar, 
    diga 'dame', 'quiero', 'llevo' o acepte la oferta.
    Registra la orden en el sistema y reserva el stock.
    """
    return f"PEDIDO CREADO EXITOSAMENTE: {cantidad}x {producto}. ID de rastreo: #9921"

tools = [consultar_inventario, crear_pedido]
print("Herramientas de Venta listas")

Herramientas de Venta listas


In [ ]:
# modelo Gemini (Vertex AI) con herramientas
llm = ChatVertexAI(model="gemini-2.5-flash", temperature=0.7)

llm_con_herramientas = llm.bind_tools(tools)

def agente_de_ventas(mensaje_usuario, historial=[]):
    print(f"Usuario: {mensaje_usuario}")
    
    # preparar mensajes con la personalidad
    mensajes = [SystemMessage(content=SALES_SYSTEM_PROMPT)] + historial + [HumanMessage(content=mensaje_usuario)]
    
    # primera llamada al Cerebro
    respuesta_ai = llm_con_herramientas.invoke(mensajes)
    
    # lógica de decisión
    if respuesta_ai.tool_calls:
        print(f"(Pensamiento): El usuario pide algo real. Usaré herramienta: {respuesta_ai.tool_calls[0]['name']}")
        
        # ejecutar la herramienta
        tool_call = respuesta_ai.tool_calls[0]
        tool_function = {
            "consultar_inventario": consultar_inventario,
            "crear_pedido": crear_pedido
        }[tool_call["name"]]
        
        resultado_tool = tool_function.invoke(tool_call)
        print(f"(Herramienta): Resultado -> {resultado_tool}")
        
        # volver al LLM con el dato obtenido para que formule la respuesta final
        mensajes.append(respuesta_ai) # Añadimos la intención de llamar tool
        mensajes.append(ToolMessage(content=str(resultado_tool), tool_call_id=tool_call["id"]))
        
        respuesta_final = llm_con_herramientas.invoke(mensajes)
        print(f"Alex (Vendedor): {respuesta_final.content}\n")
        return respuesta_final.content
        
    else:
        # Si no necesita herramienta, responde directo (Charla/Persuasión)
        print(f"Alex (Vendedor): {respuesta_ai.content}\n")
        return respuesta_ai.content

/tmp/ipykernel_68274/3643861965.py:3: DeprecationWarning: Use [`ChatGoogleGenerativeAI`][langchain_google_genai.ChatGoogleGenerativeAI] instead.
  llm = ChatVertexAI(model="gemini-2.5-flash", temperature=0.7)


In [ ]:
print("--- INICIO DE DEMO ---")
agente_de_ventas("Hola, busco zapatos para correr, pero no sé si gastar mucho.")

agente_de_ventas("¿Tienen Nike?")

agente_de_ventas("Ok, me convenciste. Dame 2 pares de Nike.")

--- INICIO DE DEMO ---
Usuario: Hola, busco zapatos para correr, pero no sé si gastar mucho.
(Pensamiento): El usuario pide algo real. Usaré herramienta: consultar_inventario
(Herramienta): Resultado -> content='NO ENCONTRADO en inventario.' name='consultar_inventario' tool_call_id='19a0d559-be6d-45b4-a675-49a49c557221'
Alex (Vendedor): [{'type': 'text', 'text': 'Disculpa, parece que con la descripción "zapatos para correr" no encuentro resultados directos en nuestro inventario. A veces los tenemos listados como "zapatillas deportivas" o "calzado para running".\n\nPermíteme buscar bajo un término más general para asegurarme de darte las mejores opciones.\n\n', 'thought_signature': 'CvIOAY89a1+O+XQ9eTMFNDMaCeTto9QDRPrS5AdBbunjSxbFq1kY1WTEhwHSWOzYxqWnpRqxzcZ+Siz9YVe46n3XjtnzOyGZqdsD0C00AosmGy9OLhdT8/rhOADauQQB2oI29W1rmVRM+u0a043TnyA7/ot++zamUagAPFzEoTD8okN1YfUwRUMIbbGfo3cfl02GvoaFIR6AgQTwglS2CmIQ08v02hH1SgYo6oI1mXvZQjwq2JB4uGptJCwvdmt4ARgj3monrCs+vpo9RNzZihhzZBEkuR5N0YlqR/XAdiSiKA3MMJU2T

'¡Excelente elección! Tu pedido de 2 pares de Nike ha sido creado exitosamente. Tu número de rastreo es #9921.\n\n¿Necesitas unos calcetines a juego o quizás un kit de limpieza para mantener tus nuevos Nike impecables?'